# 🪑 Ergonomics Chair: Posture Classification with LSTM
This updated notebook adds LSTM-based classification to detect posture types like upright, slouched, and leaning from joint positions.

In [ ]:
!pip install mediapipe opencv-python matplotlib pandas scikit-learn tensorflow

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape
from google.colab import files
from google.colab.patches import cv2_imshow

## 📤 Upload Subject Video

In [ ]:
uploaded = files.upload()

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
video_path = list(uploaded.keys())[0]
cap = cv2.VideoCapture(video_path)
landmarks_all = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if results.pose_landmarks:
        joints = []
        for lm in results.pose_landmarks.landmark:
            joints.extend([lm.x, lm.y, lm.z])
        landmarks_all.append(joints)
cap.release()
landmarks_array = np.array(landmarks_all)

## 🏷️ Manually Label Postures (0: Upright, 1: Leaned, 2: Slouched)

In [ ]:
# Example: Assign fake labels for demo purposes
num_samples = len(landmarks_array)
y_labels = np.zeros(num_samples)
y_labels[int(num_samples/3):int(2*num_samples/3)] = 1
y_labels[int(2*num_samples/3):] = 2

## 🤖 Train LSTM for Posture Classification

In [ ]:
X = landmarks_array
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_seq = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])
y = pd.get_dummies(y_labels).values
X_train, X_test, y_train, y_test = train_test_split(X_seq, y, test_size=0.2)

model = Sequential([
    LSTM(64, input_shape=(1, X_scaled.shape[1])),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))